In [1]:
import pickle
from srn import *
from tqdm import tqdm
path = '../data/'

In [2]:
with open(path+'val_scene_gt.pkl', 'rb') as f:
    S = pickle.load(f)

In [3]:
with open(path+'val_question_gt.pkl', 'rb') as f:
    Q = pickle.load(f)

In [4]:
device = 'cpu'
ERR = []
for s_idx,s in tqdm(enumerate(S)):
    # build net
    connection = scene2connection(s).transpose(0,1)
    net = SRN(connection).to(device)
    
    for q_idx,q in enumerate(Q[s_idx]):
        # make stimuli
        stimuli = program2stimuli(q['program'], connection.shape[0]).to(device)
        stimuli.unsqueeze_(2);
        # reasoning
        mem, spike= net(stimuli)
        mem.squeeze_().to('cpu')
        spike.squeeze_().to('cpu')
        # get prediction
        pred = spike2answer(mem, spike, q['program'][-1])
        if pred != q['answer']:
            ERR.append([s_idx,q_idx])

15000it [05:42, 43.76it/s]


In [5]:
len(ERR)

0